In [1]:
funcs = pyspark.sql.functions
types = pyspark.sql.types

In [2]:
bike = spark.read.parquet('/data/citibike.parquet')

In [3]:
bike.show(5)

+-------------+----------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+------------+----------+------+-----------------+---------------+
|trip_duration|      start_time|       stop_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|   user_type|birth_year|gender|start_taxizone_id|end_taxizone_id|
+-------------+----------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+------------+----------+------+-----------------+---------------+
|          801|1481563027000000|1481563829000000|            3002|"South End Ave & ...|             40.711512|     -74.01575600000001|     

In [4]:
bike = (bike
    .withColumn('start_time', 
                    funcs.from_unixtime((bike.start_time/1000000).cast(types.IntegerType()))
                    .cast(types.TimestampType()))
    .withColumn('stop_time', 
                    funcs.from_unixtime((bike.stop_time/1000000).cast(types.IntegerType()))
                    .cast(types.TimestampType()))        
#     .withColumn('start_time', 
#                 (funcs.substring(bike.start_time, 1, 20)).cast(types.TimestampType()))
#     .withColumn('stop_time', bike.stop_time.cast(types.TimestampType())) \
    .withColumn('start_taxizone_id', bike.start_taxizone_id.cast(types.FloatType())) 
    .withColumn('end_taxizone_id', bike.end_taxizone_id.cast(types.FloatType())) 
        )

In [5]:
bike.show(5)

+-------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+------------+----------+------+-----------------+---------------+
|trip_duration|          start_time|           stop_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|   user_type|birth_year|gender|start_taxizone_id|end_taxizone_id|
+-------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+------------+----------+------+-----------------+---------------+
|          801|2016-12-12 12:17:...|2016-12-12 12:30:...|            3002|"South End Ave & ...|             40.7115

In [6]:
bike.registerTempTable('bike')

In [7]:
spark.sql('select * from bike LIMIT 10').toPandas()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,start_taxizone_id,end_taxizone_id
0,801,2016-12-12 12:17:07,2016-12-12 12:30:29,3002,"""South End Ave & Liberty St""",40.711512,-74.015756,346,"""Bank St & Hudson St""",40.736529,-74.006180,25788,"""Subscriber""",1976.0,1,13.0,158.0
1,767,2016-12-12 12:17:11,2016-12-12 12:29:59,3224,"""W 13 St & Hudson St""",40.739974,-74.005139,236,"""St Marks Pl & 2 Ave""",40.728419,-73.987140,25773,"""Subscriber""",1973.0,1,249.0,79.0
2,883,2016-12-12 12:17:14,2016-12-12 12:31:58,3263,"""Cooper Square & E 7 St""",40.729236,-73.990868,127,"""Barrow St & Hudson St""",40.731724,-74.006744,20572,"""Subscriber""",1963.0,1,79.0,158.0
3,427,2016-12-12 12:17:16,2016-12-12 12:24:24,484,"""W 44 St & 5 Ave""",40.755003,-73.980144,492,"""W 33 St & 7 Ave""",40.750200,-73.990931,25620,"""Subscriber""",1985.0,1,161.0,186.0
4,478,2016-12-12 12:17:19,2016-12-12 12:25:18,519,"""Pershing Square North""",40.751873,-73.977706,526,"""E 33 St & 5 Ave""",40.747659,-73.984907,25019,"""Subscriber""",1974.0,2,170.0,164.0
5,662,2016-12-12 12:17:20,2016-12-12 12:28:22,3140,"""1 Ave & E 78 St""",40.771404,-73.953517,3294,"""E 91 St & Park Ave""",40.783502,-73.955327,16099,"""Subscriber""",1969.0,1,141.0,236.0
6,1366,2016-12-12 12:17:20,2016-12-12 12:40:07,427,"""Bus Slip & State St""",40.701907,-74.013942,514,"""12 Ave & W 40 St""",40.760875,-74.002777,23359,"""Subscriber""",1977.0,1,12.0,246.0
7,609,2016-12-12 12:17:23,2016-12-12 12:27:33,519,"""Pershing Square North""",40.751873,-73.977706,2003,"""1 Ave & E 18 St""",40.733812,-73.980544,22003,"""Subscriber""",1992.0,1,170.0,224.0
8,1009,2016-12-12 12:17:21,2016-12-12 12:34:11,323,"""Lawrence St & Willoughby St""",40.692362,-73.986317,531,"""Forsyth St & Broome St""",40.718939,-73.992663,22674,"""Subscriber""",1973.0,2,65.0,148.0
9,1018,2016-12-12 12:17:22,2016-12-12 12:34:21,3260,"""Mercer St & Bleecker St""",40.727064,-73.996621,296,"""Division St & Bowery""",40.714131,-73.997047,25510,"""Subscriber""",1970.0,1,114.0,148.0


In [8]:
bike.sort('start_time').repartition(32, 'start_station_id').sortWithinPartitions('start_station_id', 'start_time') \
    .write.parquet('/data/citibike_spark.parquet', compression='snappy', mode='overwrite')

In [9]:
subway = spark.read.parquet('/data/subway.parquet')

In [10]:
subway.show(5)

+------+------+----------+-------+--------+--------+----------------+-----------+-------------+-----------+
|    ca|  unit|       scp|station|linename|division|         endtime|description|cumul_entries|cumul_exits|
+------+------+----------+-------+--------+--------+----------------+-----------+-------------+-----------+
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|1356145200000000|  "REGULAR"|      3922274|    1352720|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|1356159600000000|  "REGULAR"|      3922288|    1352730|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|1356174000000000|  "REGULAR"|      3922352|    1352824|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|1356188400000000|  "REGULAR"|      3922544|    1352881|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|1356202800000000|  "REGULAR"|      3922944|    1352967|
+------+------+----------+-------+--------+--------+----------------+-----------+-------------+-----------+
only showing top 5 rows



In [11]:
subway = \
    subway.withColumn('endtime', funcs.from_unixtime((subway.endtime/1000000).cast(types.IntegerType())) \
                      .cast(types.TimestampType()))

In [12]:
subway.show(5)

+------+------+----------+-------+--------+--------+--------------------+-----------+-------------+-----------+
|    ca|  unit|       scp|station|linename|division|             endtime|description|cumul_entries|cumul_exits|
+------+------+----------+-------+--------+--------+--------------------+-----------+-------------+-----------+
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|2012-12-21 22:00:...|  "REGULAR"|      3922274|    1352720|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|2012-12-22 02:00:...|  "REGULAR"|      3922288|    1352730|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|2012-12-22 06:00:...|  "REGULAR"|      3922352|    1352824|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|2012-12-22 10:00:...|  "REGULAR"|      3922544|    1352881|
|"A002"|"R051"|"02-00-00"| "NULL"|  "NULL"|  "NULL"|2012-12-22 14:00:...|  "REGULAR"|      3922944|    1352967|
+------+------+----------+-------+--------+--------+--------------------+-----------+-------------+-----

In [13]:
subway = subway.repartition(32, 'ca', 'unit', 'scp').sortWithinPartitions('ca', 'unit', 'scp', 'endtime')
subway.write.parquet('/data/subway_spark.parquet', compression='snappy', mode='overwrite')

In [14]:
taxi = spark.read.parquet('/data/all_trips_unprocessed.parquet')

In [15]:
taxi.show(5)

+----------------+----------------+------------------+-------------------+---------+-----+-----------+---------------------+-------+---------------+------------+----------------+---------------+----------------+------------------+------------+------------------+----------+------------+------------+-------------+---------+---------+
|dropoff_datetime|dropoff_latitude| dropoff_longitude|dropoff_taxizone_id|ehail_fee|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type| pickup_datetime|pickup_latitude|pickup_longitude|pickup_taxizone_id|rate_code_id|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|trip_distance|trip_type|vendor_id|
+----------------+----------------+------------------+-------------------+---------+-----+-----------+---------------------+-------+---------------+------------+----------------+---------------+----------------+------------------+------------+------------------+----------+------------+------------+-------------+---------+---------

In [16]:
taxi = (
    taxi.withColumn('dropoff_datetime', 
                    funcs.from_unixtime((taxi.dropoff_datetime/1000000).cast(types.IntegerType()))
                    .cast(types.TimestampType())) \
    .withColumn('pickup_datetime', 
                    funcs.from_unixtime((taxi.pickup_datetime/1000000).cast(types.IntegerType()))
                .cast(types.TimestampType())) \
    .withColumn('dropoff_taxizone_id', taxi.dropoff_taxizone_id.cast(types.IntegerType())) \
    .withColumn('pickup_taxizone_id', taxi.pickup_taxizone_id.cast(types.IntegerType())) \
    .withColumn('dropoff_latitude', taxi.dropoff_latitude.cast(types.FloatType())) \
    .withColumn('dropoff_longitude', taxi.dropoff_longitude.cast(types.FloatType())) \
    .withColumn('ehail_fee', taxi.ehail_fee.cast(types.FloatType())) \
    .withColumn('extra', taxi.extra.cast(types.FloatType())) \
    .withColumn('fare_amount', taxi.fare_amount.cast(types.FloatType())) \
    .withColumn('improvement_surcharge', taxi.improvement_surcharge.cast(types.FloatType())) \
    .withColumn('mta_tax', taxi.mta_tax.cast(types.FloatType())) \
    .withColumn('pickup_latitude', taxi.pickup_latitude.cast(types.FloatType())) \
    .withColumn('pickup_longitude', taxi.pickup_longitude.cast(types.FloatType())) \
    .withColumn('tip_amount', taxi.tip_amount.cast(types.FloatType())) \
    .withColumn('tolls_amount', taxi.tolls_amount.cast(types.FloatType())) \
    .withColumn('total_amount', taxi.total_amount.cast(types.FloatType())) \
    .withColumn('trip_distance', taxi.trip_distance.cast(types.FloatType())) \
    .withColumn('passenger_count', taxi.passenger_count.cast(types.IntegerType())) \
    .withColumn('rate_code_id', taxi.rate_code_id.cast(types.IntegerType())) \
    .withColumn('trip_id', funcs.monotonically_increasing_id())
    )

In [17]:
taxi.show(5)

+--------------------+----------------+-----------------+-------------------+---------+-----+-----------+---------------------+-------+---------------+------------+--------------------+---------------+----------------+------------------+------------+------------------+----------+------------+------------+-------------+---------+---------+-------+
|    dropoff_datetime|dropoff_latitude|dropoff_longitude|dropoff_taxizone_id|ehail_fee|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|     pickup_datetime|pickup_latitude|pickup_longitude|pickup_taxizone_id|rate_code_id|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|trip_distance|trip_type|vendor_id|trip_id|
+--------------------+----------------+-----------------+-------------------+---------+-----+-----------+---------------------+-------+---------------+------------+--------------------+---------------+----------------+------------------+------------+------------------+----------+------------+---------

In [18]:
taxi.sort('pickup_datetime').repartition(401, 'pickup_datetime') \
    .sortWithinPartitions('pickup_taxizone_id') \
    .write \
    .parquet('/data/all_trips_spark.parquet', compression='snappy', mode='overwrite')